## Import data

In [1]:
import json

# Load the JSON file
with open('/Users/minhphuong/Downloads/TVs-all-merged.json', 'r') as file:
    data = json.load(file)


## Data cleaning

In [2]:

import pandas as pd
import re

def clean_text(text):
    # Standardize 'inch' representations
    text = re.sub(r"(?i)\b(inch|inches|”|''|-inch|\"|”)\b", "inch", text)

    # Standardize 'hertz' (hz) representations
    text = re.sub(r"(?i)\b(hertz|hz|-hz)\b", "hz", text)

    # Standardize 'pounds' (lbs) representations
    text = re.sub(r"(?i)\b(\d+(\.\d+)?)\s*(pounds|lbs|lb)\b", r"\1lbs", text)

    # Standardize 'year' and 'days' representations
    text = re.sub(r"(?i)\b(\d+)\s*(year|years|yr|yrs)\b", r"\1year", text)  # Convert to 'Nyear'
    text = re.sub(r"(?i)\b(\d+)\s*(day|days)\b", r"\1days", text)  # Convert to 'Ndays'

    # Convert power values like '5W + 5W' to '10W'
    matches = re.findall(r"(\d+(?:\.\d+)?)W", text)
    if matches:
        total_power = sum(float(num) for num in matches)
        text = re.sub(r"(\d+(?:\.\d+)?W(?:\s*\+\s*\d+(?:\.\d+)?W)*)", f"{int(total_power) if total_power.is_integer() else total_power}W", text)

    # Convert text to lowercase
    text = text.lower()

    # Remove non-alphanumeric characters (except for units like 'inch', 'hz', and 'lbs')
    text = re.sub(r"[^a-z0-9\s]", "", text)

    # Remove spaces and non-alphanumeric tokens before units (e.g., '23 inch' -> '23inch', '33.3 lbs' -> '33.3lbs')
    text = re.sub(r"(\d+(\.\d+)?)\s*(inch|hz|lbs|year|days)", r"\1\3", text)

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text




# Flattening and cleaning the hierarchical data structure
def flatten_data(data):
    products = []
    for model_id, product_list in data.items():
        for product in product_list:
            # Add 'modelID' to the product dictionary (retain for evaluation)
            product['modelID'] = model_id

            # Clean the title of each product
            if 'title' in product:
                product['cleaned_title'] = clean_text(product['title'])
            
            # Clean each value in the key-value pairs under 'featuresMap' after sorting the keys
            if 'featuresMap' in product:
                cleaned_values = []
                # Sort the keys of featuresMap to ensure a consistent order
                for key in sorted(product['featuresMap']):
                    value = product['featuresMap'][key]
                    # # Handle "Yes" values by replacing with the key
                    if value == "Yes":
                        cleaned_value = clean_text(key)
                        cleaned_values.append(cleaned_value)
                    # Skip key-value pairs where value is "No" or "0"
                    elif value != "No" and value != "0":
                        cleaned_value = clean_text(str(value))  # Convert value to string and clean it
                        cleaned_values.append(cleaned_value)
                
                # Add cleaned values to the product
                product['cleaned_featuresMap'] = cleaned_values

            # Append the cleaned product to the flattened list
            products.append(product)
    
    return products



flattened_products = flatten_data(data)
cleaned_data = [(product['modelID'], product['cleaned_title'], product['cleaned_featuresMap'], product['shop']) for product in flattened_products]
df_cleaned = pd.DataFrame(cleaned_data, columns=['modelID', 'cleaned_title', 'cleaned_featuresMap', 'shop'])
df_cleaned['cleaned_featuresMap'] = df_cleaned['cleaned_featuresMap'].apply(lambda x: ' '.join(x))



# Ensure each value in `cleaned_featuresMap` is treated as a string before joining
df_cleaned['combined_text'] = df_cleaned['cleaned_title'] + ' ' + df_cleaned['cleaned_featuresMap']


# Remove all words that do not contain numbers from 'combined_text'
df_cleaned['combined_text'] = df_cleaned['combined_text'].apply(lambda x: ' '.join(word for word in x.split() if re.search(r'\d', word)))




In [3]:

# Function to remove words longer than 6 characters and duplicates
def clean_combined_key_text(text):
    words = text.split()
    seen = set()
    unique_words = []
    for word in words:
        if len(word) <= 6 and word not in seen:  # Check length and uniqueness
            seen.add(word)
            unique_words.append(word)
    return ' '.join(unique_words)

# Apply the function to the 'combined_key_text' column
df_cleaned['combined_text'] = df_cleaned['combined_text'].apply(clean_combined_key_text)



## Functions

In [8]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from hashlib import sha1
import math


def get_shingles(text, char_ngram):
    # Tokenize the text into words
    tokens = text.split()
    # Normalize by sorting tokens alphabetically
    normalized_tokens = ' '.join(sorted(tokens))
    # Generate token-based shingles
    if char_ngram is None:  # Use word-level shingling if no n-gram size is provided
        return {normalized_tokens}
    # Generate character-level shingles
    if len(normalized_tokens) < char_ngram:
        return {normalized_tokens}
    return set(normalized_tokens[head:head + char_ngram] for head in range(len(normalized_tokens) - char_ngram + 1))



def compute_set_signature(shingle_set, hash_functions):
    set_sig = []
    for h_funct in hash_functions:
        min_hash = math.inf
        for el in shingle_set:
            h = h_funct.get_hash_value(el)
            if h < min_hash:
                min_hash = h
        # Ensure min_hash is updated correctly
        if min_hash == math.inf:
            raise ValueError("min_hash was not updated; check hash function and shingle set.")
        set_sig.append(min_hash)
    return set_sig


def get_signature_matrix(documents, k, hash_functions):
    signature_matrix = []
    for doc in documents:
        shingles = get_shingles(doc, k)
        signature = compute_set_signature(shingles, hash_functions)
        signature_matrix.append(signature)
    return signature_matrix



class HashFunction:
    def __init__(self, p=2147483647):  # Using a large prime number as `p`
        self.a = random.randint(1, p - 1)  # Random value for `a` in range [1, p-1]
        self.b = random.randint(0, p - 1)  # Random value for `b` in range [0, p-1]
        self.p = p

    def get_hash_value(self, x):
        # Compute the hash value using the formula: h_a,b(x) = (a + b * x) % p
        return (self.a + self.b * int(sha1(x.encode('utf8')).hexdigest(), 16)) % self.p

def get_signature_matrix_bands(sig_matrix, bands_nr, r):
    # r = int(sign_len / bands_nr)  # Number of rows per band
    bands = {i: [] for i in range(bands_nr)}  # Initialize bands dictionary
    for signature in sig_matrix:
        for i in range(bands_nr):
            idx = i * r
            # band_tuple = tuple(signature[idx:idx + r])
            # bands[i].append(band_tuple)
            bands[i].append(' '.join(str(x) for x in signature[idx:idx+r]) )       
    return bands




def get_band_buckets(band, hash_function):
    buckets = {}
    for doc_id, band_values in enumerate(band):
        # Convert the band tuple to a string to use with the custom hash function
        band_str = ''.join(map(str, band_values))
        band_hash = hash_function.get_hash_value(band_str)
        if band_hash not in buckets:
            buckets[band_hash] = [doc_id]
        else:
            buckets[band_hash].append(doc_id)
    return buckets




def get_candidates_list(buckets):
    candidates = set()
    # Buckets is a dictionary containing key=bucket, value=list of doc_ids that hashed to the bucket
    for bucket, candidate_list in buckets.items():
        if len(candidate_list) > 1:
            for i in range(0, len(candidate_list) - 1):
                for j in range(i + 1, len(candidate_list)):  
                    pair = tuple(sorted((candidate_list[i], candidate_list[j])))
                    candidates.add(pair)
    return candidates  # A set of couples, each couple is a candidate pair





## Generate candidate pairs using LSH then detect duplicates 

In [ ]:
from hashlib import sha1
import numpy as np
from itertools import combinations
from sentence_transformers import SentenceTransformer, util
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist, squareform

# Bootstrapping with LSH implementation
n_bootstraps = 5
sign_len = 100  # Length of signature (number of rows)
k = 5  # Shingle length
num_hash_functions = sign_len  # Number of hash functions
random_b_values = [1, 7, 14, 19, 21, 24, 26, 31, 38, 40, 45, 49, 53, 60, 64, 73, 80, 87, 91, 100]

fractions_of_comparisons = []
pair_completeness_values = []
bands_rows_comparisons = []
num_comparisons_values = []  # To store num_comparisons for each number of bands
pair_quality_values = []  # To store pair quality for each number of bands
f1_measure_values = []  # To store F1-measure for each number of bands
f1star_measure_values = []  # To store F1* measure for each number of bands

# To store b and r values for each configuration
b_values = []
r_values = []

# Generate hash functions
hash_functions = [HashFunction() for _ in range(num_hash_functions)]

# Iterate over number of bands (b)
for b in random_b_values:  # From 1 to 100 bands
    r = max(1, sign_len // b)  # Calculate rows per band, ensuring at least 1 row per band

    # Store `b` and `r` values for the current configuration
    b_values.append(b)
    r_values.append(r)

    pair_completeness_bootstrap = []
    fraction_comparisons_bootstrap = []
    num_comparisons_per_bands = []  # Store num_comparisons for each bootstrap
    pair_quality_bootstrap = []  # Store pair quality for each bootstrap
    precision_bootstrap = []
    recall_bootstrap = []


    for _ in range(n_bootstraps):

        # Bootstrap sample (100% of the data, with replacement)
        sample = df_cleaned.sample(frac=1, replace=True, random_state=13 + _).reset_index(drop=True)

        original_indices = sample.index.tolist()
        sample['product_descript'] = sample.apply(lambda row: (row['shop'], row['cleaned_title']), axis=1)

        # Identify unique products
        unique_product_indices = []
        seen_products = set()
        for idx, product_id in enumerate(sample['product_descript']):
            if product_id not in seen_products:
                seen_products.add(product_id)
                unique_product_indices.append(idx)

        # Create unique_products DataFrame
        bootstrap_df = sample.iloc[unique_product_indices].reset_index(drop=True)

        # Generate signature matrix for unique products
        unique_documents = bootstrap_df['combined_text'].tolist()
        bootstrap_sig_matrix = get_signature_matrix(unique_documents, k, hash_functions)

        # Generate bands
        bands = get_signature_matrix_bands(bootstrap_sig_matrix, b, r)

        # Generate candidate pairs for each band and aggregate them
        candidate_pairs = set()

        for band_id, band in enumerate(bands.values()):
            # Select a hash function for this band (cyclic if there are more bands than hash functions)
            hash_function = hash_functions[band_id % len(hash_functions)]
            buckets = get_band_buckets(band, hash_function)
            candidates = get_candidates_list(buckets)
            candidate_pairs.update(candidates)

        # Perform similarity checks on candidate pairs
        detected_duplicates = []

        # Load a pre-trained Sentence Transformer model
        model = SentenceTransformer("all-MiniLM-L6-v2")

        # Generate embeddings for all rows in the dataframe
        texts = bootstrap_df['combined_text']
        embeddings = model.encode(texts.values)

        # Initialize an infinite dissimilarity matrix
        # Initialize the dissimilarity matrix with a large value (10^8) for all non-candidate pairs
        dissimilarity_matrix = np.full((len(bootstrap_df), len(bootstrap_df)), 10**8)


        # Compute dissimilarities for candidate pairs
        for pair in candidate_pairs:
            idx1, idx2 = pair

            # Extract combined text for both indices
            text1 = bootstrap_df.loc[idx1, 'combined_text']
            text2 = bootstrap_df.loc[idx2, 'combined_text']

            # Compute similarity using embeddings
            embedding1 = embeddings[list(bootstrap_df.index).index(idx1)]
            embedding2 = embeddings[list(bootstrap_df.index).index(idx2)]
            similarity = util.cos_sim(embedding1, embedding2).item()

            # Convert similarity to dissimilarity
            dissimilarity = 1 - similarity

            # Update the dissimilarity matrix for the candidate pair
            dissimilarity_matrix[idx1, idx2] = dissimilarity
            dissimilarity_matrix[idx2, idx1] = dissimilarity  # Symmetric matrix

        # Enforce constraints for same shop and different brands
        for i in range(len(bootstrap_df)):
            for j in range(i + 1, len(bootstrap_df)):
                # Check if the pair does not meet conditions (e.g., different brands)
                if (
                    bootstrap_df.loc[i, 'shop'] == bootstrap_df.loc[j, 'shop'] and
                    bootstrap_df.loc[i, 'Brand'] != bootstrap_df.loc[j, 'Brand']
                ):
                    dissimilarity_matrix[i, j] = 10**8
                    dissimilarity_matrix[j, i] = 10**8  # Symmetric


        # Ensure it's in condensed form for clustering
        condensed_dissimilarity = squareform(dissimilarity_matrix, checks=False)

        # Perform average linkage clustering
        linkage_matrix = linkage(condensed_dissimilarity, method='average')

        # Define a threshold for clustering (adjust as needed)
        threshold = 0.6

        # Extract cluster labels
        clusters = fcluster(linkage_matrix, threshold, criterion='distance')

        # Assign clusters to the dataframe
        bootstrap_df['cluster'] = clusters

        # Identify duplicate pairs based on clusters
        detected_duplicates = []
        for i in range(len(bootstrap_df)):
            for j in range(i + 1, len(bootstrap_df)):
                if bootstrap_df.loc[i, 'cluster'] == bootstrap_df.loc[j, 'cluster']:
                    detected_duplicates.append((i, j))


        # Calculate total number of true duplicate pairs in the bootstrap dataset
        true_duplicate_pairs = 0
        duplicate_groups = bootstrap_df['modelID'].value_counts()
        for count in duplicate_groups:
            if count > 1:
                true_duplicate_pairs += count * (count - 1) / 2

        # Calculate metrics for the bootstrap sample
        num_comparisons = len(candidate_pairs)
        total_possible_comparisons = len(bootstrap_sig_matrix) * (len(bootstrap_sig_matrix) - 1) / 2
        duplicates_found = len([pair for pair in detected_duplicates if bootstrap_df.loc[pair[0], 'modelID'] == bootstrap_df.loc[pair[1], 'modelID']])
        pair_completeness = duplicates_found / true_duplicate_pairs if true_duplicate_pairs > 0 else 0
        recall = pair_completeness
        precision = duplicates_found / len(detected_duplicates) 
        fraction_of_comparisons = num_comparisons / total_possible_comparisons if total_possible_comparisons > 0 else 0
        pair_quality = duplicates_found / num_comparisons if num_comparisons > 0 else 0

        # Store metrics for the bootstrap
        pair_completeness_bootstrap.append(pair_completeness)
        fraction_comparisons_bootstrap.append(fraction_of_comparisons)
        num_comparisons_per_bands.append(num_comparisons)
        pair_quality_bootstrap.append(pair_quality)
        precision_bootstrap.append(precision)
        recall_bootstrap.append(recall)


    # Average over all bootstraps
    avg_pair_completeness = np.mean(pair_completeness_bootstrap)
    avg_pair_quality = np.mean(pair_quality_bootstrap)
    avg_precision = np.mean(precision_bootstrap)
    avg_recall = np.mean(recall_bootstrap)

    # Calculate F1-measure using the formula: F1 = 2 * PQ * PC / (PQ + PC)
    f1_measure = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1star_measure = (2 * avg_pair_quality * avg_pair_completeness) / (avg_pair_quality + avg_pair_completeness) if (avg_pair_quality + avg_pair_completeness) > 0 else 0

    # Store metrics for this number of bands
    pair_completeness_values.append(avg_pair_completeness)
    fractions_of_comparisons.append(np.mean(fraction_comparisons_bootstrap))
    bands_rows_comparisons.append(np.mean(num_comparisons_per_bands))
    num_comparisons_values.append(np.mean(num_comparisons_per_bands))
    pair_quality_values.append(avg_pair_quality)
    f1_measure_values.append(f1_measure)
    f1star_measure_values.append(f1star_measure)

# Print b, r, num_comparisons, pair quality, and F1-measure values for each configuration
for i, b in enumerate(b_values):
    print(f"Number of Bands: {b}, Rows per Band: {r_values[i]}, num_comparisons: {num_comparisons_values[i]}, pair_quality: {pair_quality_values[i]}, F1-measure: {f1_measure_values[i]}, F1star-measure: {f1star_measure_values[i]}")


## Evaluate performance

In [ ]:
import matplotlib.pyplot as plt

# Generalized plotting function for subplots
def plot_metric_subplot(ax, fractions, values, xlabel, ylabel, title, xlim=None, ylim=None, color='b'):
    ax.plot(fractions, values, marker='', linestyle='-', color=color)  # Smooth line
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    if xlim:
        ax.set_xlim(xlim)
    if ylim:
        ax.set_ylim(ylim)
    ax.grid(True)
    ax.set_aspect('equal', adjustable='box')  # Make each subplot area square

# Create subplots for 3 plots in 1 row
fig, axs = plt.subplots(1, 4, figsize=(18, 6))  # 1 row, 3 columns

# Plot Pair Completeness
plot_metric_subplot(
    axs[0],
    fractions_of_comparisons,
    pair_completeness_values,
    xlabel='Fraction of Comparisons',
    ylabel='Pair Completeness',
    title='Pair Completeness',
    xlim=(0, 1),
    ylim=(0, 1),
    color='b'
)

# Plot Pair Quality
plot_metric_subplot(
    axs[1],
    fractions_of_comparisons,
    pair_quality_values,
    xlabel='Fraction of Comparisons',
    ylabel='Pair Quality',
    title='Pair Quality',
    xlim=(0, 0.06),
    ylim=(0, 0.06),
    color='b'
)

# Plot F1-Measure
plot_metric_subplot(
    axs[2],
    fractions_of_comparisons,
    f1_measure_values,
    xlabel='Fraction of Comparisons',
    ylabel='F1-Measure',
    title='F1-Measure',
    xlim=(0, 0.04),
    ylim=(0, 0.04),
    color='b'
)

# Plot F1*-Measure
plot_metric_subplot(
    axs[3],
    fractions_of_comparisons,
    f1star_measure_values,
    xlabel='Fraction of Comparisons',
    ylabel='F1*-Measure',
    title='F1*-Measure',
    xlim=(0, 0.04),
    ylim=(0, 0.04),
    color='b'
)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()
